In [ ]:
from paddleocr import PaddleOCR

ocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=False)


In [37]:
from PIL import Image
import numpy as np
import cv2
import os
from pathlib import Path


# For images with multiple gauges
def identify_gauges(image_path):
    img = cv2.imread(image_path)
    
    
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    
    lower_orange = np.array([0, 100, 100])
    upper_orange = np.array([30, 255, 255])
    
   
    mask = cv2.inRange(hsv, lower_orange, upper_orange)
    
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    
    gauge_readings = []
    
    # Process each potential gauge
    for i, contour in enumerate(contours):
        
        x, y, w, h = cv2.boundingRect(contour)
        
        
        if w < 20 or h < 20:
            continue
            
        # Extract region of interest (ROI)
        roi = img[y:y+h, x:x+w]
        
        
        temp_path = f'gauge_{i}.jpg'
        cv2.imwrite(temp_path, roi)
        
        
        
        result = ocr.ocr(temp_path)
        
        if result and result[0]:
            for line in result[0]:
                try:
                    text = line[1][0] 
                    confidence = line[1][1]
                    
                    # Only keep numerical readings with high confidence
                    if any(char.isdigit() for char in text) and confidence > 0.5:
                        gauge_readings.append({
                            'gauge_id': i,
                            'position': (x, y),
                            'text': text,
                            'confidence': confidence
                        })
                        
                        # Draw rectangle and text on original image
                        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                        cv2.putText(img, text, (x, y-10), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                except Exception as e:
                    continue
    
    
    cv2.imwrite('annotated_gauges.jpg', img)
    
    return gauge_readings

# For images one gauge
def read_digital_display(image_path):
    img = cv2.imread(image_path)
    
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    
    _, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
    
    
    result = ocr.ocr(image_path)
    
    
    readings = []
    
    if result and result[0]:
        
        numerical_readings = []
        for line in result[0]:
            try:
                text = line[1][0]
                confidence = line[1][1]
                box = line[0]
                
                
                x = min(p[0] for p in box)
                y = min(p[1] for p in box)
                
                # Filter based on position and content
                if any(char.isdigit() for char in text):
                    numerical_readings.append({
                        'text': ''.join(filter(str.isdigit, text)),
                        'confidence': confidence,
                        'x': x,
                        'y': y
                    })
            except Exception as e:
                continue
        
        # Sort by x-coordinate (left to right)
        numerical_readings.sort(key=lambda x: x['x'])
        
        # Combine digits
        if numerical_readings:
            combined_text = ''.join(r['text'] for r in numerical_readings)
            avg_confidence = sum(r['confidence'] for r in numerical_readings) / len(numerical_readings)
            
            readings.append({
                'text': combined_text,
                'confidence': avg_confidence
            })
    
    return readings

# For processing multiple images
def process_folder(folder_path):
    # Get all image files
    image_extensions = ('.png', '.jpg', '.jpeg', '.gif', '.bmp')
    image_files = [
        f for f in Path(folder_path).iterdir() 
        if f.suffix.lower() in image_extensions
    ]
    
    # Process each image and store results
    results = {}
    
    for img_path in image_files:
        print(f"\nProcessing: {img_path.name}")
        try:
            # Read image
            img = cv2.imread(str(img_path))
            if img is None:
                print(f"Could not read image: {img_path}")
                continue
                
            # Convert to HSV color space
            hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            
            # Define orange color range
            lower_orange = np.array([0, 100, 100])
            upper_orange = np.array([30, 255, 255])
            
            # Create mask for orange regions
            mask = cv2.inRange(hsv, lower_orange, upper_orange)
            
            # Find contours
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Process based on number of contours
            if len(contours) < 3:
                readings = read_digital_display(str(img_path))
            else:
                readings = identify_gauges(str(img_path))
                
            results[img_path.name] = readings
            
            # Display image (if in Jupyter notebook)
            image = Image.open(img_path)
            display(image)
            
            # Print results for this image
            if readings:
                for reading in readings:
                    print(f"Text: {reading['text']}, Confidence: {reading['confidence']:.2f}")
            else:
                print("No text detected")
                
        except Exception as e:
            print(f"Error processing {img_path.name}: {e}")
            results[img_path.name] = []
    
    return results

In [ ]:
folder_path = "./images"
results = process_folder(folder_path)